# **DeepFace - Passport vs Profile Image - Invoke and run from APEX**

- Runs deep face pipeline (detecting faces, cropping image, flatten image, extract face, etc)
- Cross-checks detected face(s) from two images
- Extracts predicted agen and predicted gender from both

## **1. Imports**

In [ ]:
# #1.st define bucket and namespace where to store
# !odsc conda init -b conda_environment_yolov5 -n frqap2zhtzbe -a resource_principal

# #2.nd publish conda env to bucket
# !odsc conda publish -s tensorflow28_p38_gpu_v1 --force

In [1]:
from deepface import DeepFace
import matplotlib.pyplot as plt
import cv2
import fsspec
from PIL import Image
from io import BytesIO
from ads.model.framework.tensorflow_model import TensorFlowModel
from ads.common.model_metadata import UseCaseType
from ads.common.model_artifact import ModelArtifact
from ads.common.model_export_util import prepare_generic_model
import os
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, Convolution2D
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import keras.models
import tensorflow
from keras.preprocessing.image import load_img, img_to_array
from keras.applications.vgg16 import preprocess_input
import numpy as np
from keras.models import model_from_json, Model

In [ ]:
#!pip uninstall opencv-python

In [ ]:
#!pip install opencv-python-headless

## **2. Test Deepface locally**

### This will download the first time the model weights locally in " /home/datascience /.deepface/weights"

In [ ]:
##
## Can not use Deepface like the below in model deployment. It will try and download the model and store locally. This will fail.
##

from deepface import DeepFace 
passport_input = "./example_images/bob_pass.jpg"            #my passport
profile_image_input = "./example_images/bob_pf_1.jpg"       #profile image

result = DeepFace.verify(passport_input, profile_image_input)
result

## **3. Create model artifacts**

In [12]:
#path to artifacts and conda slug
path_to_artifacts = '/home/datascience/model_artifacts'

#create default artifacts
artifact = prepare_generic_model(path_to_artifacts, fn_artifact_files_included=False, force_overwrite=True, inference_conda_env="tensorflow28_p38_gpu_v1", inference_python_version="3.7")

loop1:   0%|          | 0/4 [00:00<?, ?it/s]

The inference conda environment is tensorflow28_p38_gpu_v1 and the Python version is 3.7.


In [13]:
%%writefile "{path_to_artifacts}/runtime.yaml"

# Model runtime environment
MODEL_ARTIFACT_VERSION: '3.0'
MODEL_DEPLOYMENT:
  INFERENCE_CONDA_ENV:
    INFERENCE_ENV_PATH: oci://conda_environment_yolov5@frqap2zhtzbe/conda_environments/gpu/TensorFlow 2.8 for GPU on Python 3.8/1.0/tensorflow28_p38_gpu_v1
    INFERENCE_ENV_SLUG: tensorflow28_p38_gpu_v1
    INFERENCE_ENV_TYPE: published
    INFERENCE_PYTHON_VERSION: '3.8'
MODEL_PROVENANCE:
  PROJECT_OCID: ocid1.datascienceproject.oc1.eu-frankfurt-1.amaaaaaangencdyaik5ssdqk4as2bhldxprh7vnqpk7yycsm7vymd344cgua
  TENANCY_OCID: ocid1.tenancy.oc1..aaaaaaaabu5fgingcjq3vc7djuwsdcutdxs4gsws6h4kfoldqpjuggxprgoa
  TRAINING_COMPARTMENT_OCID: ocid1.compartment.oc1..aaaaaaaae3n6r6hrjipbap2hojicrsvkzatrtlwvsyrpyjd7wjnw4za3m75q
  TRAINING_CONDA_ENV:
    TRAINING_ENV_PATH: oci://conda_environment_yolov5@frqap2zhtzbe/conda_environments/gpu/TensorFlow 2.8 for GPU on Python 3.8/1.0/tensorflow28_p38_gpu_v1
    TRAINING_ENV_SLUG: tensorflow28_p38_gpu_v1
    TRAINING_ENV_TYPE: published
    TRAINING_PYTHON_VERSION: '3.8'
  TRAINING_REGION: eu-frankfurt-1
  TRAINING_RESOURCE_OCID: ocid1.datasciencenotebooksession.oc1.eu-frankfurt-1.amaaaaaangencdyacxmsz5ycch762wjc54udhibtl3m4nacuaf7shrvyoktq
  USER_OCID: ocid1.saml2idp.oc1..aaaaaaaar3ydw5hoiob7dfjzoom2dvbhqkkd5fat6m7upe72emlsxhsfrbfa/bob.peulen@oracle.com
  VM_IMAGE_INTERNAL_ID: NB1480-DCGPU131-VMP64-VMA1585-BI681

Overwriting /home/datascience/model_artifacts/runtime.yaml


## **4. Get model weights and test locally**

In [ ]:
# copy the weights to the model artifact folder
!cp /home/datascience/.deepface/weights/vgg_face_weights.h5 /home/datascience/model_artifacts
!cp /home/datascience/.deepface/weights/gender_model_weights.h5 /home/datascience/model_artifacts
!cp /home/datascience/.deepface/weights/age_model_weights.h5 /home/datascience/model_artifacts

In [ ]:
#### functions
def preprocess_image(image_path):
    img = load_img(image_path, target_size=(224, 224))
    
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = preprocess_input(img)
    return img

def findCosineSimilarity(source_representation, test_representation):
    a = np.matmul(np.transpose(source_representation), test_representation)
    b = np.sum(np.multiply(source_representation, source_representation))
    c = np.sum(np.multiply(test_representation, test_representation))
    return 1 - (a / (np.sqrt(b) * np.sqrt(c)))

def findEuclideanDistance(source_representation, test_representation):
    euclidean_distance = source_representation - test_representation
    euclidean_distance = np.sum(np.multiply(euclidean_distance, euclidean_distance))
    euclidean_distance = np.sqrt(euclidean_distance)
    return euclidean_distance


In [ ]:
### define model
model = Sequential()
model.add(ZeroPadding2D((1,1),input_shape=(224,224, 3)))
model.add(Convolution2D(64, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))
 
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(128, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))
 
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))
 
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))
 
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))
 
model.add(Convolution2D(4096, (7, 7), activation='relu'))
model.add(Dropout(0.5))
model.add(Convolution2D(4096, (1, 1), activation='relu'))
model.add(Dropout(0.5))
model.add(Convolution2D(2622, (1, 1)))
model.add(Flatten())
model.add(Activation('softmax'))

In [ ]:
#load weights in model
model.load_weights('/home/datascience/model_artifacts/vgg_face_weights.h5')

#define input and ouput layer
vgg_face_descriptor = Model(inputs=model.layers[0].input, outputs=model.layers[-2].output)

In [ ]:
# #save model locally in model artifacts
# vgg_face_descriptor.save("/home/datascience/model_artifacts/model_artifacts")

# model = keras.models.load_model("/home/datascience/model_artifacts/model_artifacts")

In [ ]:
#test on two images
passport_input = "./example_images/bob_pass.jpg"
profile_image_input = "./example_images/bob_pf_1.jpg"

#define distance and sensitivity
epsilon = 0.60 #cosine similarity
#epsilon = 120 #euclidean distance
 
img1_representation = vgg_face_descriptor.predict(preprocess_image(passport_input))[0,:]
img2_representation = vgg_face_descriptor.predict(preprocess_image(profile_image_input))[0,:]
 
cosine_similarity = findCosineSimilarity(img1_representation, img2_representation)
#euclidean_distance = findEuclideanDistance(img1_representation, img2_representation)
 
if(cosine_similarity < epsilon):
    print("Same person")
else:
    print("Not same person!")

-----------

## **5. Score.py**

In [14]:
%%writefile "{path_to_artifacts}/score.py"

########################################
########################################
########################################
########################################
########################################
#################### Imports
#################### 
####################
####################

import glob      
import os
import base64
import uuid
import io
from deepface import DeepFace
import matplotlib.pyplot as plt
import cv2
import fsspec
from PIL import Image
from io import BytesIO
from ads.model.framework.tensorflow_model import TensorFlowModel
from ads.common.model_metadata import UseCaseType
from ads.common.model_artifact import ModelArtifact
from ads.common.model_export_util import prepare_generic_model
import os
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, Convolution2D
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import keras.models
import tensorflow
from keras.preprocessing.image import load_img, img_to_array
from keras.applications.vgg16 import preprocess_input
import numpy as np
from keras.models import model_from_json, Model
import shutil

########################################
########################################
########################################
########################################
########################################
#################### Model
#################### 
####################
####################

def load_model():                                        ### check to load_model instead of in predict
    
#     #load weights in model
#     #model.load_weights('./.deepface/weights/vgg_face_weights.h5')   #in model deployment
#     model.load_weights('./model_artifacts/.deepface/weights/vgg_face_weights.h5')   #locally notebook

#     #define input and ouput layer
#     vgg_face_descriptor = Model(inputs=model.layers[0].input, outputs=model.layers[-2].output)
    
#     return vgg_face_descriptor


    class DummyModel:
        def __init__(self):
            pass
    return DummyModel()

########################################
########################################
########################################
########################################
########################################
#################### Predict
#################### 
####################
####################

def predict(data, model=load_model()):
    
    #get the base64 images from the payload
    passport_data = data['data']['passport_data']
    profile_data = data['data']['profile_data']
    
    #input image folder
    path_input_image_locally = "/home/datascience/images" 
    
    #delete folder when exists
    if os.path.exists(path_input_image_locally):
        shutil.rmtree(path_input_image_locally)
    
    #make as new folder
    if not os.path.exists(path_input_image_locally):         
        os.makedirs(path_input_image_locally)
        
       
    ##### decoding of passport
    img_bytes_p = io.BytesIO(base64.b64decode(passport_data.encode('utf-8')))
    passport_image = Image.open(img_bytes_p).resize((224, 224))  #same as input layer? and pre processing
    
    #save image locally     
    passport_image = passport_image.save(path_input_image_locally + "/passport_image.jpg")
    
    ##### decoding of profile
    img_bytes_d = io.BytesIO(base64.b64decode(profile_data.encode('utf-8')))
    profile_image = Image.open(img_bytes_d).resize((224, 224))  #same as input layer? and pre processing
    
    #save image locally     
    profile_image = profile_image.save(path_input_image_locally + "/profile_image.jpg")
    
    #make prediction. Deepface tries to load from externally. But changed the path below. Verify is a pipeline of steps. Also extracts face from image, 3d to 2d, etc.
    result_deepface = DeepFace.verify(path_input_image_locally + "/passport_image.jpg", img2_path = path_input_image_locally + "/profile_image.jpg")
    
    #extract key stats
    verified = result_deepface['verified']   # need to convert this to text. json doesn't like True or False
    distance = result_deepface['distance']
    
    verified_text = str()
    #turn bool (false true) into text
    if verified == True:
        verified_text = "Same person"
    else:
        verified_text = "Different persons"
        
#     ######
#     ###### output faces
#     detected_face_profile = DeepFace.detectFace("./profile_image.jpg", detector_backend = 'opencv')
#     detected_face = detected_face * 255
#     cv2.imwrite("face.jpg", detected_face[:, :, ::-1])

    #### 
    #### age and gender
    objs_profile = DeepFace.analyze(img_path = path_input_image_locally + "/profile_image.jpg", actions = ['age', 'gender'])
    objs_passport = DeepFace.analyze(img_path = path_input_image_locally + "/passport_image.jpg", actions = ['age', 'gender'])
    
    age_prof = objs_profile[0]['age']
    gender_prof = objs_profile[0]['dominant_gender']
    age_pass = objs_passport[0]['age']
    gender_pass = objs_passport[0]['dominant_gender']

    return {'prediction_deepface': {'verified': verified_text, 'distance': distance, 'age_prof': age_prof, 'gender_prof':gender_prof, 'age_pass':age_pass, 'gender_pass': gender_pass}}

Overwriting /home/datascience/model_artifacts/score.py


## **Create a payload. Two images in one base64 string**

In [4]:
## first passport
## second profile

import glob      
import os
import base64
import uuid
import io

file_path="/home/datascience/1_projects/deepface/testx_images/"

payload_string = str()
full_payload_string = str()
counter = 0

for images in os.listdir(file_path):
    
    counter += 1
    
    input_path=os.path.join(file_path, images)
    print(input_path)
    
    with open(input_path, "rb") as image2string:
        converted_string = base64.b64encode(image2string.read()).decode('ascii')
               
        #add payload to full string
        
        if counter == 1:
            payload1 = json.dumps(converted_string)
            json_payload1 = json.loads(payload1)
            
        else: #if counter is 2
            payload2 = json.dumps(converted_string)
            json_payload2 = json.loads(payload2)

payload_json = {'data':{'passport_data': json_payload1,'profile_data': json_payload2}}

/home/datascience/1_projects/deepface/testx_images/bob_pass.jpg
/home/datascience/1_projects/deepface/testx_images/bob_pf_1.jpg


In [30]:
#!rm -r /home/datascience/images_test/.ipynb_checkpoints

## **Model Catalog**

In [15]:
#all should be passed
artifact.introspect()

['.ipynb_checkpoints', 'score.py', 'runtime.yaml', 'test_json_output.json', 'gender_model_weights.h5', 'age_model_weights.h5', 'vgg_face_weights.h5']


,Test key,Test name,Result,Message
0,runtime_env_path,Check that field MODEL_DEPLOYMENT.INFERENCE_ENV_PATH is set,Passed,
1,runtime_env_python,Check that field MODEL_DEPLOYMENT.INFERENCE_PYTHON_VERSION is set to a value of 3.6 or higher,Passed,
2,runtime_path_exist,Check that the file path in MODEL_DEPLOYMENT.INFERENCE_ENV_PATH is correct.,Passed,
3,runtime_version,Check that field MODEL_ARTIFACT_VERSION is set to 3.0,Passed,
4,runtime_yaml,"Check that the file ""runtime.yaml"" exists and is in the top level directory of the artifact directory",Passed,
5,score_load_model,Check that load_model() is defined,Passed,
6,score_predict,Check that predict() is defined,Passed,
7,score_predict_arg,Check that all other arguments in predict() are optional and have default values,Passed,
8,score_predict_data,"Check that the only required argument for predict() is named ""data""",Passed,
9,score_py,"Check that the file ""score.py"" exists and is in the top level directory of the artifact directory",Passed,


In [16]:
# Saving the model artifact to the model catalog. 
catalog_entryx = artifact.save(display_name='deepface_oda_v8', description='deepface_oda_v8_changed_payload', timeout=600)

catalog_entryx.id

loop1:   0%|          | 0/5 [00:00<?, ?it/s]

'ocid1.datasciencemodel.oc1.eu-frankfurt-1.amaaaaaangencdyagoq64zzokto5oxxl65m32yjueb2yggu36uy4pq57rj5a'

## **Model Deployment invoke**

In [6]:
%%time
import requests
import oci
from oci.signer import Signer
import json

# REST API
uri = f"https://modeldeployment.eu-frankfurt-1.oci.customer-oci.com/ocid1.datasciencemodeldeployment.oc1.eu-frankfurt-1.amaaaaaangencdya3p3wmufctj23w3t4o25xla77eryywml7q2oti6sai7cq/predict"

# Using Resource principal to authenticate against the model endpoint 
auth = oci.auth.signers.get_resource_principals_signer()
    

response = requests.post(uri, json=payload_json, auth=auth)
print(response)

print(json.loads(response.content))

<Response [200]>
{'prediction_deepface': {'verified': 'Same person', 'distance': 0.3519110140477171, 'age_prof': 27, 'gender_prof': 'Man', 'age_pass': 24, 'gender_pass': 'Man'}}
CPU times: user 53 ms, sys: 2.39 ms, total: 55.4 ms
Wall time: 3.34 s


In [19]:
x

{'prediction_deepface': {'verified': 'Same person',
  'distance': 0.3519110140477171,
  'age_prof': 27,
  'gender_prof': 'Man',
  'age_pass': 24,
  'gender_pass': 'Man'}}

## **Gradio**

In [23]:
import requests
import oci
from oci.signer import Signer
import json



def full_function(img_1, img_2):
    
    
    payload_string = str()
    full_payload_string = str()
    counter = 0

    for image in [img_1, img_2]:

        counter += 1

#         input_path=os.path.join(file_path, images)
#         print(input_path)

        with open(image, "rb") as image2string:
            converted_string = base64.b64encode(image2string.read()).decode('ascii')

            #add payload to full string

            if counter == 1:
                payload1 = json.dumps(converted_string)
                json_payload1 = json.loads(payload1)

            else: #if counter is 2
                payload2 = json.dumps(converted_string)
                json_payload2 = json.loads(payload2)

    payload_json = {'data':{'passport_data': json_payload1,'profile_data': json_payload2}}
    
    

    # REST API
    uri = f"https://modeldeployment.eu-frankfurt-1.oci.customer-oci.com/ocid1.datasciencemodeldeployment.oc1.eu-frankfurt-1.amaaaaaangencdya3p3wmufctj23w3t4o25xla77eryywml7q2oti6sai7cq/predict"

    # Using Resource principal to authenticate against the model endpoint 
    auth = oci.auth.signers.get_resource_principals_signer()


    response = requests.post(uri, json=payload_json, auth=auth)
    
    x = json.loads(response.content)

    result_person = x['prediction_deepface']['verified']
    age_img_1 = x['prediction_deepface']['age_prof']
    gender_img_1 = x['prediction_deepface']['gender_prof']
    age_img_2 = x['prediction_deepface']['age_pass']
    gender_img_2 = x['prediction_deepface']['gender_pass']
    

    return result_person, age_img_1, gender_img_1, age_img_2, gender_img_2


In [ ]:
import gradio as gr

desc = "Compare persons!"

with gr.Blocks() as demo: 
     
    img_1 = gr.Image(type="filepath", info="Upload image 1")
    img_2 = gr.Image(type="filepath", info="Upload image 2")

    result_person = gr.Text(label="Result from DeepFace")
    age_img_1 = gr.Text(label="Predicted age image 1")
    gender_img_1 = gr.Text(label="Predicted gender image 1")
    age_img_2 = gr.Text(label="Predicted age image 2")
    gender_img_2 = gr.Text(label="Predicted gender image 2")

    submit_btn = gr.Button("Run DeepFace")


gr.Interface(fn=full_function, inputs=[img_1, img_2], outputs=[result_person, age_img_1, gender_img_1, age_img_2, gender_img_2], title=desc).launch(share=True, debug=True)

Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://aef4e977479340a6e0.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces


----

----

# **Overwriting the deep face files to load model locally, which are stored in the artifacts**

## **Change the .py file that tries to get the model to locall directory**

The below slightly changes the .py files that is in the Deepface library. So that it loads the model from local directory and not tries to load from hub.

In [ ]:
%%writefile "/home/datascience/conda/tensorflow28_p38_gpu_v1/lib/python3.8/site-packages/deepface/basemodels/VGGFace.py"

import os
import gdown
import tensorflow as tf
from deepface.commons import functions

# ---------------------------------------

tf_version = int(tf.__version__.split(".", maxsplit=1)[0])

if tf_version == 1:
    from keras.models import Model, Sequential
    from keras.layers import (
        Convolution2D,
        ZeroPadding2D,
        MaxPooling2D,
        Flatten,
        Dropout,
        Activation,
    )
else:
    from tensorflow.keras.models import Model, Sequential
    from tensorflow.keras.layers import (
        Convolution2D,
        ZeroPadding2D,
        MaxPooling2D,
        Flatten,
        Dropout,
        Activation,
    )

# ---------------------------------------


def baseModel():
    model = Sequential()
    model.add(ZeroPadding2D((1, 1), input_shape=(224, 224, 3)))
    model.add(Convolution2D(64, (3, 3), activation="relu"))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(64, (3, 3), activation="relu"))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(128, (3, 3), activation="relu"))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(128, (3, 3), activation="relu"))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(256, (3, 3), activation="relu"))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(256, (3, 3), activation="relu"))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(256, (3, 3), activation="relu"))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(512, (3, 3), activation="relu"))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(512, (3, 3), activation="relu"))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(512, (3, 3), activation="relu"))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(512, (3, 3), activation="relu"))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(512, (3, 3), activation="relu"))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(512, (3, 3), activation="relu"))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

    model.add(Convolution2D(4096, (7, 7), activation="relu"))
    model.add(Dropout(0.5))
    model.add(Convolution2D(4096, (1, 1), activation="relu"))
    model.add(Dropout(0.5))
    model.add(Convolution2D(2622, (1, 1)))
    model.add(Flatten())
    model.add(Activation("softmax"))

    return model


# url = 'https://drive.google.com/uc?id=1CPSeum3HpopfomUEK1gybeuIVoeJT_Eo'


def loadModel(
    url="https://github.com/serengil/deepface_models/releases/download/v1.0/vgg_face_weights.h5",
):

    model = baseModel()

    # -----------------------------------

    home = functions.get_deepface_home()
    output = "./vgg_face_weights.h5"                                   ################# change this path!!!!! ################# ####################################################################
    # locally: "./model_artifacts/.deepface/weights/vgg_face_weights.h5" 
    # original: output = home + "/.deepface/weights/vgg_face_weights.h5"
    # in model deployment: "./vgg_face_weights.h5" 

    if os.path.isfile(output) != True:
        print("vgg_face_weights.h5 will be downloaded...")
        gdown.download(url, output, quiet=False)

    # -----------------------------------

    model.load_weights(output)

    # -----------------------------------

    # TO-DO: why?
    vgg_face_descriptor = Model(inputs=model.layers[0].input, outputs=model.layers[-2].output)

    return vgg_face_descriptor


In [ ]:
%%writefile "/home/datascience/conda/tensorflow28_p38_gpu_v1/lib/python3.8/site-packages/deepface/extendedmodels/Age.py"

import os
import gdown
import numpy as np
import tensorflow as tf
from deepface.basemodels import VGGFace
from deepface.commons import functions

# ----------------------------------------
# dependency configurations

tf_version = int(tf.__version__.split(".", maxsplit=1)[0])

if tf_version == 1:
    from keras.models import Model, Sequential
    from keras.layers import Convolution2D, Flatten, Activation
elif tf_version == 2:
    from tensorflow.keras.models import Model, Sequential
    from tensorflow.keras.layers import Convolution2D, Flatten, Activation

# ----------------------------------------


def loadModel(
    url="https://github.com/serengil/deepface_models/releases/download/v1.0/age_model_weights.h5",
):

    model = VGGFace.baseModel()

    # --------------------------

    classes = 101
    base_model_output = Sequential()
    base_model_output = Convolution2D(classes, (1, 1), name="predictions")(model.layers[-4].output)
    base_model_output = Flatten()(base_model_output)
    base_model_output = Activation("softmax")(base_model_output)

    # --------------------------

    age_model = Model(inputs=model.input, outputs=base_model_output)

    # --------------------------

    # load weights

#     home = functions.get_deepface_home()

#     if os.path.isfile(home + "/.deepface/weights/age_model_weights.h5") != True:
#         print("age_model_weights.h5 will be downloaded...")

#         output = home + "/.deepface/weights/age_model_weights.h5"
#         gdown.download(url, output, quiet=False)

    age_model.load_weights("./age_model_weights.h5")

    return age_model

    # --------------------------


def findApparentAge(age_predictions):
    output_indexes = np.array(list(range(0, 101)))
    apparent_age = np.sum(age_predictions * output_indexes)
    return apparent_age



In [ ]:
%%writefile "/home/datascience/conda/tensorflow28_p38_gpu_v1/lib/python3.8/site-packages/deepface/extendedmodels/Gender.py"

import os
import gdown
import tensorflow as tf
from deepface.basemodels import VGGFace
from deepface.commons import functions

# -------------------------------------
# pylint: disable=line-too-long
# -------------------------------------
# dependency configurations

tf_version = int(tf.__version__.split(".", maxsplit=1)[0])

if tf_version == 1:
    from keras.models import Model, Sequential
    from keras.layers import Convolution2D, Flatten, Activation
elif tf_version == 2:
    from tensorflow.keras.models import Model, Sequential
    from tensorflow.keras.layers import Convolution2D, Flatten, Activation
# -------------------------------------

# Labels for the genders that can be detected by the model.
labels = ["Woman", "Man"]


def loadModel(
    url="https://github.com/serengil/deepface_models/releases/download/v1.0/gender_model_weights.h5",
):

    model = VGGFace.baseModel()

    # --------------------------

    classes = 2
    base_model_output = Sequential()
    base_model_output = Convolution2D(classes, (1, 1), name="predictions")(model.layers[-4].output)
    base_model_output = Flatten()(base_model_output)
    base_model_output = Activation("softmax")(base_model_output)

    # --------------------------

    gender_model = Model(inputs=model.input, outputs=base_model_output)

    # --------------------------

    # load weights

#     home = functions.get_deepface_home()

#     if os.path.isfile(home + "/.deepface/weights/gender_model_weights.h5") != True:
#         print("gender_model_weights.h5 will be downloaded...")

#         output = home + "/.deepface/weights/gender_model_weights.h5"
#         gdown.download(url, output, quiet=False)

    gender_model.load_weights("./gender_model_weights.h5")

    return gender_model